In [1]:
import numpy as np
from torch.utils.data import Dataset

class ExampleDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)

In [3]:
sample_data = np.arange(0, 10)
print('The whole data:', sample_data)

dataset = ExampleDataset(sample_data)
print('Number of samples in the dataset: ', len(dataset))

The whole data: [0 1 2 3 4 5 6 7 8 9]
Number of samples in the dataset:  10


In [6]:
print(dataset[1])

1


## Import Libraries

In [331]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from torchvision.transforms import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## Get Device

In [13]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    
    return device
device = get_device()

## Load and Prepare the Data

In [12]:
df_train = pd.read_csv('mnist_train.csv')
df_test = pd.read_csv('mnist_test.csv')

In [14]:
df_train

,5,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.608,0.609,0.610,0.611,0.612,0.613,0.614,0.615,0.616,0.617
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59994,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59995,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#get the pixel values and labels
train_labels = df_train.iloc[:, 0]
train_images = df_train.iloc[:, 1:]
test_labels = df_test.iloc[:, 0]
test_images = df_test.iloc[:, 1:]

## Define the Image Transformations

In [19]:
transform = transforms.Compose(
        [transforms.ToPILImage(), 
        transforms.ToTensor(), 
        transforms.Normalize((0.5,), (0.5,))
        ])

## Prepare the Custom Dataset and DataLoaders

In [56]:
class MNISTDataset(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
    
    def __getitem__(self, i):
        data = self.X.iloc[i, :]
        data = np.asarray(data).astype(np.uint8).reshape(28, 28, 1)
        
        if self.transforms:
            data = self.transforms(data)
        if self.y is not None:
            return (data, self.y[i])
        else:
            return data
    def __len__(self):
        return (len(self.X))

In [309]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28), y

#Instead of WrappedDataLoader we can write a collate function and that will do the same job.

class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [310]:
train_data = MNISTDataset(train_images, train_labels, transform)
test_data = MNISTDataset(test_images, test_labels, transform)

In [311]:
trainloader = DataLoader(train_data, batch_size = 1024, shuffle = True)
testloader = DataLoader(test_data, batch_size = 1024, shuffle = True)

In [312]:
train_dl = WrappedDataLoader(trainloader, preprocess)
test_dl = WrappedDataLoader(testloader, preprocess)

In [313]:
train_data[0][0].shape

torch.Size([1, 28, 28])

## Define the Neural Network, Train, and Test it


In [314]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=800, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x 
model = Net().to(device)
print(net)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(5, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=50, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [315]:
model(train_data[0][0].reshape(1, 1, 28, 28))

tensor([[ 0.1246, -0.1110,  0.0522,  0.0179, -0.0353,  0.0865, -0.0161,  0.0200,
         -0.1448, -0.0728]], grad_fn=<AddmmBackward>)

In [316]:
loss_func = F.cross_entropy

In [341]:
lr = 0.5  # learning rate
epochs = 2

def get_model():
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr) 
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
    return model, optimizer, scheduler

In [343]:
model, opt, scheduler = get_model()
# model = trained_model
for epoch in range(epochs):
    total = 0
    for i, (xb, yb) in enumerate(train_dl):
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()
        if i % 25 == 0:
            print('[Epoch %d, Iter %d] Train Loss: %.3f' %
                      (epoch + 1, i, loss.item(),))
    scheduler.step()
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_dl:
            xb, yb = data[0].to(device), data[1].to(device)
            outputs = model(xb)
            valid_loss = loss_func(outputs, yb)
            _, predicted = torch.max(outputs, 1)
            total += yb.size(0)
            correct += (predicted == yb).sum().item()
    print('[Epoch %d] Val Loss '%(epoch +1), "%.2f" % float(valid_loss / len(test_dl)), 
          'Validation Accuracy: %0.3f %%' % (100 * correct / total))

[Epoch 1, Iter 0] Train Loss: 2.299
[Epoch 1, Iter 25] Train Loss: 1.317
[Epoch 1, Iter 50] Train Loss: 0.398
[Epoch 1] Val Loss  0.02 Validation Accuracy: 93.909 %
[Epoch 2, Iter 0] Train Loss: 0.223
[Epoch 2, Iter 25] Train Loss: 0.123
[Epoch 2, Iter 50] Train Loss: 0.093
[Epoch 2] Val Loss  0.01 Validation Accuracy: 96.330 %


In [326]:
# torch.save(model.state_dict(), 'pretrained_weights/model_state_dict.pt')
torch.save(model, 'pretrained_weights/model.pt')

In [327]:
trained_model = torch.load('pretrained_weights/model.pt')

In [320]:
trained_model.eval()
test(trained_model, test_dl)

Accuracy of the network on test images: 98.850 %
